<img src="https://toppng.com/uploads/preview/linkedin-logo-png-photo-116602552293wtc4qogql.png" width="20" height="20" /> [Bharath Hemachandran](https://www.linkedin.com/in/bharath-hemachandran/)

# 🧩 Phase 3: Agents + MCP (single-agent intro)

In Phase 2 you gave a model **one MCP server (Hugging Face)** so it could call tools. In this phase, you’ll see how that becomes a simple **agent**: a model that can **decide what to do next**, use tools, and work toward a goal.

<div style="background: #e1f5fe; padding: 14px; border-radius: 8px; border-left: 4px solid #0288d1;">
<strong>🎯 What you'll do:</strong> Start from the Phase 2 MCP setup, define what an <strong>agent</strong> is, run a <strong>single agent</strong> that uses tools (including the Hugging Face MCP server), inspect its tool calls step by step, and briefly compare popular agent frameworks.
</div>

### 📋 Notebook objective (table of contents)

This notebook covers:
- **Recap from Phase 2** — Reuse Groq + Hugging Face MCP setup
- **What is an agent?** — Intuition and mental model
- **Single-agent pattern** — One agent using tools (including MCP)
- **Implement a simple agent with Groq** — Step-by-step run with `responses.create`
- **Inspect tool calls (`mcp_call`)** — See how the agent actually used MCP
- **Compare popular agent frameworks** — LangChain, LlamaIndex, crewAI, AutoGen (conceptual)
- **Exercises** — Agent vs API call, inspect mcp_call, prompt vs extra tools
- **Additional reading** — Agents, tools, and MCP


## 🔧 Setup (run once)

Same as Phase 2: install **openai** and set API keys (GROQ + optional HF_TOKEN). We’ll then reuse that setup to build a very simple **agent** on top.

In [ ]:
!pip install -q openai

In [ ]:
import os
from getpass import getpass

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass("Paste your GROQ_API_KEY: ")
if not os.environ.get("HF_TOKEN"):
    tok = getpass("Paste your HF_TOKEN (or press Enter to skip): ")
    if tok:
        os.environ["HF_TOKEN"] = tok

from openai import OpenAI

def get_groq_client():
    return OpenAI(
        api_key=os.environ["GROQ_API_KEY"],
        base_url="https://api.groq.com/openai/v1",
    )

HF_TOKEN = os.environ.get("HF_TOKEN")
print("✅ Groq client ready.")

## 🔧 MCP tool (same as Phase 2)

We’ll use the <strong>same Hugging Face MCP tool</strong> from Phase 2. In this notebook, it becomes one of the tools our <strong>agent</strong> is allowed to use.

In [ ]:
mcp_tool = {
    "type": "mcp",
    "server_label": "Huggingface",
    "server_url": "https://huggingface.co/mcp",
    "server_description": "Search and access AI models from Hugging Face",
    "require_approval": "never",
}
if HF_TOKEN:
    mcp_tool["headers"] = {"Authorization": f"Bearer {HF_TOKEN}"}

print("MCP tool: Hugging Face")

## 🧩 From tools to agents (recap of Phase 2)

In **Phase 2**, you:

- Configured Groq’s `responses.create` with **one MCP tool** (Hugging Face search).
- Let the model **call that tool** when it needed more information.
- Read the final answer from `response.output_text` and inspected `mcp_call` items.

You can think of that as the **building block of an agent**:

- The **model** reasons about what it needs.
- The **client** (Groq) exposes **tools** (via MCP) that the model can call.
- The **MCP server** (Hugging Face) runs the actual work (search, read, etc.).

In this phase we put a thin “agent” wrapper around this pattern and make the idea of an agent explicit.

## 🧠 What is an "agent" in this context?

In this course, an **agent** is:

> A loop around a model that can **decide what to do next**, call **tools** (like MCP servers), and gradually work toward a **goal**, instead of just answering once.

Very simple agents can be just **one model call** that is allowed to use tools (the client handles tool calls for you). More advanced agents add things like:

- **State / memory** — The agent remembers past steps.
- **Planning** — The agent decomposes a big task into smaller actions.
- **Multiple tools** — The agent chooses between tools (search, database, code, etc.).
- **Multi-agent setups** — Several agents talk to each other (out of scope for this phase).

In this notebook we keep it simple: **one agent, one MCP server, one goal**.

## 🚀 Single-agent pattern with Groq + MCP

We’ll treat a single `responses.create` call (with tools enabled) as a **simple agent run**:

1. You give the agent a **goal** in natural language.
2. The model decides whether to call the **Hugging Face MCP tool**.
3. The MCP server runs the tool and returns results.
4. The model uses those results to produce a final answer.

You’ll see these steps reflected as `mcp_call` items inside `response.output`.

In [ ]:
client = get_groq_client()

def run_agent(task: str):
    """Very simple single-agent run using Groq + Hugging Face MCP.

    - Takes a natural-language task.
    - Lets the model decide if/when to call the MCP tool.
    - Returns the full response so we can inspect tool calls.
    """
    response = client.responses.create(
        model="openai/gpt-oss-120b",
        input=task,
        tools=[mcp_tool],
    )
    return response

# Example: give the agent a clear goal that requires Hugging Face.
example_task = (
    "Act as a model search agent. Use Hugging Face to find trending "
    "text-generation models and then summarize the top 2 in one short paragraph."
)

response = run_agent(example_task)

print("--- agent output_text ---")
print(response.output_text)

## 📋 Inspecting the agent's tool calls (`mcp_call`)

Each time the agent (model) uses the Hugging Face MCP tool, you’ll see an item in `response.output` with <code>type == "mcp_call"</code>.

We’ll print a short preview of each to see **what tools the agent chose, in what order, and with what outputs**.

In [ ]:
print("--- mcp_call steps ---")
for item in response.output:
    if getattr(item, "type", None) == "mcp_call":
        name = getattr(item, "name", "?")
        out = str(getattr(item, "output", ""))[:200]
        print(f"  {name}: {out}...")

In [ ]:
print("✅ Phase 3 complete. You now have a simple single agent using MCP.")

## ✏️ Exercises

*Use only what you learned in this phase (what an agent is, single agent + tools, mcp_call, frameworks).*

1. **Agent vs one-off API call**  
   In your own words, what makes a **single agent** (like the one we built) different from a **one-off API call** that has no tools? Answer in one or two sentences.

2. **Inspect tool usage**  
   After running the agent (the cell that asks for trending text-generation models and summarizes the top 2), look at the `mcp_call` steps in `response.output`. How many tool calls did the agent make? For each call, what is the **name** of the tool and what did it do in one short phrase (e.g. "search for models")?

3. **Prompt vs extra tools**  
   Suppose you want the agent to (a) search Hugging Face and (b) return the summary **only as bullet points**. Would you add another MCP server, or change the prompt, or both? Justify in one or two sentences using what you learned about agents and tools.

## 📚 Additional reading

**YouTube (verified)**  
- [The Model Context Protocol (MCP)](https://www.youtube.com/watch?v=CQywdSdi5iA) — Anthropic: MCP and tool integration.  
- [Model Context Protocol, clearly explained](https://www.youtube.com/watch?v=7j_NE6Pjv-E) — Multi-step agents and external tools.  
- [LangChain Agents overview](https://www.youtube.com/results?search_query=langchain+agents) — How LangChain structures tools, agents, and planning.

**Blogs / docs (popular)**  
- [Model Context Protocol](https://modelcontextprotocol.io) — Official site: protocol, servers, clients.  
- [Groq Blog – MCP and connectors](https://groq.com/blog) — Groq: MCP support and agent workflows.  
- [LangChain Agents](https://python.langchain.com/docs/modules/agents/) — Agents, tools, and tool-calling orchestration.  
- [LlamaIndex agents](https://docs.llamaindex.ai) — Index-centric agents and tool use.  
- [crewAI](https://www.crewai.com/) and [AutoGen](https://aka.ms/autogen) — Multi-agent orchestration frameworks (beyond this phase).